# Modelling Medical Insurance data using logistic regression

In [0]:
# Data processing
from pyspark.sql.functions import *

# Modeling
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [0]:
write_path = 'dbfs:/tmp/reproducible_ml_uofl/medinsurance_linearreg_class1_7_csv.delta'
medins_chrgs = spark.read.format('delta').load(write_path)

#Show basic summary stats
display(medins_chrgs.summary())

summary,age,bmi,children,region,charges,gender_cd,smoker_cd
count,1338,1338,1338,1338,1338,1338,1338
mean,39.20702541106129,30.663396860986538,1.0949177877429,null,13270.422265141257,0.4947683109118087,0.20478325859491778
stddev,14.049960379216147,6.098186911679012,1.205492739781914,null,12110.011236693992,0.5001595692843768,0.40369403754561717
min,18,15.96,0,northeast,1121.8739,0,0
25%,27,26.29,0,null,4738.2682,0,0
50%,39,30.4,1,null,9377.9047,0,0
75%,51,34.7,2,null,16657.71745,1,0
max,64,53.13,5,southwest,63770.42801,1,1


In [0]:
from pyspark.sql.window import Window
import pyspark.sql.functions as f
from pyspark.sql.functions import row_number

#create percentile ranks
medins_chrgs = medins_chrgs.select("age","bmi","children","region","charges","gender_cd","smoker_cd", 
                                   f.row_number().over(Window.partitionBy().orderBy(medins_chrgs['charges'])).alias("row_num"),
                                   f.percent_rank().over(Window.partitionBy().orderBy(medins_chrgs['charges'])).alias("percent_rank"))
#display(medins_chrgs)

#create targets for Top 5%, 10%, and 25%
medins_chrgs_new=(medins_chrgs.withColumn('top05_trgt', f.when(f.col('percent_rank') >=0.95, 1).otherwise(0))
                  .withColumn('top10_trgt', f.when(f.col('percent_rank') >=0.9, 1).otherwise(0))
                  .withColumn('top25_trgt', f.when(f.col('percent_rank') >=0.75, 1).otherwise(0))
)
medins_chrgs_new.show()
medins_chrgs_new.groupBy('top05_trgt').count().show()
medins_chrgs_new.groupBy('top10_trgt').count().show()
medins_chrgs_new.groupBy('top25_trgt').count().show()

+---+-----+--------+---------+---------+---------+---------+-------+--------------------+----------+----------+----------+
age| bmi|children| region| charges|gender_cd|smoker_cd|row_num| percent_rank|top05_trgt|top10_trgt|top25_trgt|
+---+-----+--------+---------+---------+---------+---------+-------+--------------------+----------+----------+----------+
 18|23.21| 0|southeast|1121.8739| 0| 0| 1| 0.0| 0| 0| 0|
 18|30.14| 0|southeast|1131.5066| 0| 0| 2|7.479431563201197E-4| 0| 0| 0|
 18|33.33| 0|southeast|1135.9407| 0| 0| 3|0.001495886312640...| 0| 0| 0|
 18|33.66| 0|southeast|1136.3994| 0| 0| 4|0.002243829468960359| 0| 0| 0|
 18| 34.1| 0|southeast| 1137.011| 0| 0| 5|0.002991772625280...| 0| 0| 0|
 18|34.43| 0|southeast|1137.4697| 0| 0| 6|0.003739715781600...| 0| 0| 0|
 18|37.29| 0|southeast|1141.4451| 0| 0| 7|0.004487658937920718| 0| 0| 0|
 18|41.14| 0|southeast|1146.7966| 0| 0| 8|0.005235602094240838| 0| 0| 0|
 18|43.01| 0|southeast|1149.3959| 0| 0| 9|0.005983545250560957| 0| 0| 0|
 18|53.13| 0|southeast|1163.4627| 0| 0| 10|0.006731488406881077| 0| 0| 0|
 19| 19.8| 0|southwest| 1241.565| 0| 0| 11|0.007479431563201...| 0| 0| 0|
 19| 20.3| 0|southwest| 1242.26| 0| 0| 12|0.008227374719521317| 0| 0| 0|
 19| 20.7| 0|southwest| 1242.816| 0| 0| 13|0.008975317875841436| 0| 0| 0|
 19| 27.6| 0|southwest| 1252.407| 0| 0| 14|0.009723261032161555| 0| 0| 0|
 19| 28.7| 0|southwest| 1253.936| 0| 0| 15|0.010471204188481676| 0| 0| 0|
 19| 30.4| 0|southwest| 1256.299| 0| 0| 16|0.011219147344801795| 0| 0| 0|
 19| 34.1| 0|southwest| 1261.442| 0| 0| 17|0.011967090501121914| 0| 0| 0|
 19| 34.4| 0|southwest| 1261.859| 0| 0| 18|0.012715033657442034| 0| 0| 0|
 19| 35.4| 0|southwest| 1263.249| 0| 0| 19|0.013462976813762155| 0| 0| 0|
 20|33.33| 0|southeast|1391.5287| 0| 0| 20|0.014210919970082274| 0| 0| 0|
+---+-----+--------+---------+---------+---------+---------+-------+--------------------+----------+----------+----------+
only showing top 20 rows

+----------+-----+
top05_trgt|count|
+----------+-----+
 0| 1271|
 1| 67|
+----------+-----+

+----------+-----+
top10_trgt|count|
+----------+-----+
 0| 1204|
 1| 134|
+----------+-----+

+----------+-----+
top25_trgt|count|
+----------+-----+
 0| 1003|
 1| 335|
+----------+-----+

In [0]:
# Train test split
trainDF, testDF = medins_chrgs_new.randomSplit([.65, .35], seed=42)
# Print the number of records
print(f'There are {trainDF.cache().count()} records in the training dataset.')
print(f'There are {testDF.cache().count()} records in the testing dataset.')

There are 919 records in the training dataset.
There are 419 records in the testing dataset.

In [0]:
#One hot encoding of string feature Region
trainDF.groupBy('region').count().show()

+---------+-----+
 region|count|
+---------+-----+
northeast| 222|
southeast| 261|
southwest| 224|
northwest| 212|
+---------+-----+

##Now we need to modify the categorical variable region into one-hot-encoded version
 For this we will use a pipeline to do this in one step

In [0]:
#You can also create a pipeline and do everything together in one easy fit and transform step
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
 
categoricalColumns = ["region"]
stages = [] # stages in Pipeline
for categoricalCol in categoricalColumns:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    
# Use OneHotEncoder to convert categorical variables into binary SparseVectors
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])

# Define the pipeline based on the stages created in previous steps.
pipeline = Pipeline(stages=[stringIndexer, encoder])
 
# Define the pipeline model.
transform_mdl = pipeline.fit(trainDF)
trainDF21=transform_mdl.transform(trainDF)
trainDF21.show()

+---+------+--------+---------+-----------+---------+---------+-------+--------------------+----------+----------+----------+-----------+--------------+
age| bmi|children| region| charges|gender_cd|smoker_cd|row_num| percent_rank|top05_trgt|top10_trgt|top25_trgt|regionIndex|regionclassVec|
+---+------+--------+---------+-----------+---------+---------+-------+--------------------+----------+----------+----------+-----------+--------------+
 18| 15.96| 0|northeast| 1694.7964| 0| 0| 48|0.035153328347045626| 0| 0| 0| 2.0| (3,[2],[1.0])|
 18| 17.29| 2|northeast| 12829.4551| 0| 1| 893| 0.6671652954375468| 0| 0| 0| 2.0| (3,[2],[1.0])|
 18| 21.47| 0|northeast| 1702.4553| 0| 0| 49| 0.03590127150336574| 0| 0| 0| 2.0| (3,[2],[1.0])|
 18| 21.66| 0|northeast| 14283.4594| 1| 1| 959| 0.7165295437546746| 0| 0| 0| 2.0| (3,[2],[1.0])|
 18| 22.99| 0|northeast| 1704.5681| 0| 0| 50| 0.03664921465968586| 0| 0| 0| 2.0| (3,[2],[1.0])|
 18| 23.32| 1|southeast| 1711.0268| 0| 0| 55| 0.04038893044128646| 0| 0| 0| 0.0| (3,[0],[1.0])|
 18| 24.09| 1|southeast| 2201.0971| 1| 0| 117| 0.08676140613313388| 0| 0| 0| 0.0| (3,[0],[1.0])|
 18|26.125| 0|northeast| 1708.92575| 0| 0| 54|0.039640987284966345| 0| 0| 0| 2.0| (3,[2],[1.0])|
 18| 26.18| 2|southeast| 2304.0022| 0| 0| 132| 0.09798055347793568| 0| 0| 0| 0.0| (3,[0],[1.0])|
 18|26.315| 0|northeast| 2198.18985| 1| 0| 115| 0.08526551982049364| 0| 0| 0| 2.0| (3,[2],[1.0])|
 18| 27.28| 3|southeast| 18223.4512| 1| 1| 1029| 0.768885564697083| 0| 0| 1| 0.0| (3,[0],[1.0])|
 18|28.215| 0|northeast| 2200.83085| 1| 0| 116| 0.08601346297681377| 0| 0| 0| 2.0| (3,[2],[1.0])|
 18|29.165| 0|northeast|7323.734819| 1| 0| 529| 0.3949139865370232| 0| 0| 0| 2.0| (3,[2],[1.0])|
 18| 29.37| 1|southeast| 1719.4363| 0| 0| 57|0.041884816753926704| 0| 0| 0| 0.0| (3,[0],[1.0])|
 18| 30.03| 1|southeast| 1720.3537| 0| 0| 58| 0.04263275991024682| 0| 0| 0| 0.0| (3,[0],[1.0])|
 18|30.305| 0|northeast| 2203.73595| 1| 0| 119| 0.08825729244577413| 0| 0| 0| 2.0| (3,[2],[1.0])|
 18| 31.13| 0|southeast| 1621.8827| 1| 0| 28| 0.02019446522064323| 0| 0| 0| 0.0| (3,[0],[1.0])|
 18| 31.68| 2|southeast| 34303.1672| 0| 1| 1197| 0.8945400149588631| 0| 0| 1| 0.0| (3,[0],[1.0])|
 18| 31.73| 0|northeast| 33732.6867| 0| 1| 1191| 0.8900523560209425| 0| 0| 1| 2.0| (3,[2],[1.0])|
 18| 31.92| 0|northeast| 2205.9808| 1| 0| 120| 0.08900523560209424| 0| 0| 0| 2.0| (3,[2],[1.0])|
+---+------+--------+---------+-----------+---------+---------+-------+--------------------+----------+----------+----------+-----------+--------------+
only showing top 20 rows

In [0]:
trainDF21.printSchema()

root
-- age: integer (nullable = true)
-- bmi: double (nullable = true)
-- children: integer (nullable = true)
-- region: string (nullable = true)
-- charges: double (nullable = true)
-- gender_cd: integer (nullable = true)
-- smoker_cd: integer (nullable = true)
-- row_num: integer (nullable = true)
-- percent_rank: double (nullable = true)
-- top05_trgt: integer (nullable = false)
-- top10_trgt: integer (nullable = false)
-- top25_trgt: integer (nullable = false)
-- regionIndex: double (nullable = false)
-- regionclassVec: vector (nullable = true)

In [0]:
# Linear regression expect a vector input
vecAssembler = VectorAssembler(inputCols=['age','bmi','children','gender_cd','smoker_cd','regionclassVec'], outputCol="features")
vecTrainDF = vecAssembler.transform(trainDF21)

# Take a look at the data
#display(vecTrainDF)

In [0]:
testDF21=transform_mdl.transform(testDF) #do the data transformation using saved parameters from training
vecTestDF = vecAssembler.transform(testDF21) #do the feature transformation using vector assembler

In [0]:
# Create linear regression
lr = LogisticRegression(featuresCol="features", labelCol="top10_trgt")
# Fit the linear regresssion model
lrModel = lr.fit(vecTrainDF)
predict_train = lrModel.transform(vecTrainDF)

# Make predictions on testing dataset
predict_test = lrModel.transform(vecTestDF) #make predictions using the trained model

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

eval = BinaryClassificationEvaluator(labelCol = "top10_trgt")

auc_train = eval.evaluate(predict_train)
print(auc_train)

auc_test = eval.evaluate(predict_test)
print(auc_test)


0.9859703417150278
0.9949208443271769

In [0]:
print(lrModel.summary.areaUnderROC)
print("Coefficients: \n" + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))

0.9858284977433917
Coefficients: 
DenseMatrix([[ 0.06274767, 0.50424427, 0.30393565, 0.34952187, 10.16871613,
 -0.06492295, 0.80747231, 0.39489357]])
Intercept: [-28.831158721646403]

In [0]:
import pandas as pd
import time 
import sys
from pyspark.sql.window import *
from pyspark.sql.types import * 
from pyspark.sql.functions import *
import pyspark.sql.functions as f

def get_model_stats(print_lable,dataset,evaluator,mbr_id,target):
  print(print_lable)
  AUROC=evaluator.evaluate(dataset)
  
  tot_event=dataset.count()
  tot_target=dataset.select(target).filter(f.col(target)==1).count()
  inc_rate=float(tot_target)/float(tot_event)
  print('Incidence rate in dataset: ',inc_rate)
  print('AUROC: ',AUROC)
  

  def mbr_prob1(prob_vec):
    return float(prob_vec[1]) #Get the second value from probability vector of the prediction since that is probability of target 1
  def mbr_prob2(rawp_vec):
    prob_vec = np.exp(rawp_vector)/(1+ np.exp(rawp_vec)) #this is the same as the logit function e**x/1+e**x
    return float(prob_vec[1])
  get_prob_1= udf(mbr_prob1, FloatType())
  get_prob_2= udf(mbr_prob2, FloatType())

  try:
    df_plot_tmp = dataset.withColumn('prob_score', get_prob_1('probability'))
  except:
    df_plot_tmp = dataset.withColumn('prob_score', get_prob_2('rawPrediction'))
  df_reg1=(df_plot_tmp.select(mbr_id,"rawPrediction","probability","prob_score",target))

  #Create Percentiles based on the predicted probability
  df_reg_12=  df_reg1.sort(col("prob_score").desc())
  df_reg_12 = df_reg_12.withColumn("new_column",lit("ABC"))
  w = Window().partitionBy("new_column").orderBy(col("prob_score").desc())  
  df_reg_12 = df_reg_12.withColumn("row_num",row_number().over(w)).drop("new_column")
  df_reg2 = (df_reg_12
             .withColumn("score_pctl", f.percent_rank().over(Window.orderBy(f.col("row_num"))))
             .withColumn("pctl_cat", f.when(f.col("score_pctl")==1, f.lit(100)).otherwise(f.floor(f.col("score_pctl")*100) + 1))
             )
  df_reg22 = (df_reg2
              .groupBy("pctl_cat")
              .agg(f.mean(target).alias("pctl_target_rate"),\
                   (f.sum(target)).alias("pctl_target_total"),\
                   (f.count(mbr_id)).alias("total_count"),\
                   (f.mean("prob_score")).alias("mean_predicted_prob")
                  ,)
              .orderBy(f.col("pctl_cat"))
              )

  #Now let's create the cumulative percentiles
  cum_sum = (df_reg22
              .withColumn('cum_target', f.sum(df_reg22.pctl_target_total).over(Window.partitionBy().orderBy().rowsBetween(-sys.maxsize, 0)))
              .withColumn('cum_total', f.sum(df_reg22.total_count).over(Window.partitionBy().orderBy().rowsBetween(-sys.maxsize, 0)))
              .withColumn('cum_target_cap_rate',f.col("cum_target")/f.col("cum_total"))
              .withColumn('cum_target_cap_of_total',((f.col("cum_target")/tot_target))*100)
            )
  return cum_sum
  

In [0]:
cump_cap_details = get_model_stats("Logistic Regression",predict_test,eval,"row_num","top10_trgt")
display(cump_cap_details)

pctl_cat,pctl_target_rate,pctl_target_total,total_count,mean_predicted_prob,cum_target,cum_total,cum_target_cap_rate,cum_target_cap_of_total
1,1.0,5,5,0.9949912548065185,5,5,1.0,12.5
2,1.0,4,4,0.9856249392032623,9,9,1.0,22.5
3,1.0,4,4,0.9693315327167511,13,13,1.0,32.5
4,1.0,4,4,0.9497404098510742,17,17,1.0,42.5
5,1.0,4,4,0.8989026248455048,21,21,1.0,52.5
6,1.0,5,5,0.8513014674186706,26,26,1.0,65.0
7,1.0,4,4,0.8082806766033173,30,30,1.0,75.0
8,0.75,3,4,0.6996464878320694,33,34,0.9705882352941176,82.5
9,0.75,3,4,0.5973019450902939,36,38,0.9473684210526315,90.0
10,0.25,1,4,0.5096063390374184,37,42,0.8809523809523809,92.5


In [0]:
# Create a glm with binomial and logit to mimic logictic regression
from pyspark.ml.regression import GeneralizedLinearRegression

glr = GeneralizedLinearRegression(featuresCol="features", labelCol="top10_trgt",family="binomial", link="logit", maxIter=10, regParam=0.3)

# Fit the model
model = glr.fit(vecTrainDF)
predict_train_glr = model.transform(vecTrainDF)

eval_glr = BinaryClassificationEvaluator(rawPredictionCol = "prediction", labelCol = "top10_trgt")

auc_train_glr = eval_glr.evaluate(predict_train_glr)
print(auc_train_glr)
print(model.summary)

# Print the coefficients and intercept for generalized linear regression model
predict_test_glr = model.transform(vecTestDF) #make predictions using the trained model
auc_test_glr = eval_glr.evaluate(predict_test_glr)
print(auc_test_glr)

0.9748549323017478
Coefficients:
 Feature Estimate Std Error T Value P Value
 (Intercept) -9.0562 0.8484 -10.6738 0.0000
 age 0.0153 0.0088 1.7329 0.0831
 bmi 0.1681 0.0212 7.9345 0.0000
 children 0.0983 0.0910 1.0803 0.2800
 gender_cd -0.1776 0.1666 -1.0662 0.2863
 smoker_cd 1.9956 0.1682 11.8676 0.0000
regionclassVec_so... 0.0036 0.1761 0.0205 0.9836
regionclassVec_so... 0.0846 0.1811 0.4669 0.6406
regionclassVec_no... 0.0033 0.1842 0.0181 0.9855

(Dispersion parameter for binomial family taken to be 1.0000)
 Null deviance: 606.6779 on 910 degrees of freedom
Residual deviance: 300.8168 on 910 degrees of freedom
AIC: 318.8168
0.9757255936675463

##This code is used for regularization

In [0]:
# Create linear regression
lr2 = LogisticRegression(featuresCol="features", labelCol="top10_trgt", regParam=0, elasticNetParam=0.0)
# Fit the linear regresssion model
lrModel2 = lr2.fit(vecTrainDF)
predict_train2 = lrModel2.transform(vecTrainDF)

# Make predictions on testing dataset
predict_test2 = lrModel2.transform(vecTestDF) #make predictions using the trained model

In [0]:
eval = BinaryClassificationEvaluator(labelCol = "top10_trgt")

auc_train2 = eval.evaluate(predict_train2)
print(auc_train2)
auc_test2 = eval.evaluate(predict_test2)
print(auc_test2)

print("Training AUROC", lrModel.summary.areaUnderROC)
print("Coefficients: \n" + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))

0.9859703417150278
0.9949208443271769
Training AUROC 0.9858284977433917
Coefficients: 
DenseMatrix([[ 0.06274767, 0.50424427, 0.30393565, 0.34952187, 10.16871613,
 -0.06492295, 0.80747231, 0.39489357]])
Intercept: [-28.831158721646403]

In [0]:
# Create lasso logistic regression
lr_lasso = LogisticRegression(featuresCol="features", labelCol="top10_trgt", regParam=0.1, elasticNetParam=1)
# Fit the linear regresssion model
lrModel_lasso = lr_lasso.fit(vecTrainDF)
predict_train31 = lrModel_lasso.transform(vecTrainDF)

# Make predictions on testing dataset
predict_test31 = lrModel_lasso.transform(vecTestDF) #make predictions using the trained model

auc_train31 = eval.evaluate(predict_train31)
print(auc_train31)
auc_test31 = eval.evaluate(predict_test31)
print(auc_test31)

print("Training AUROC", lrModel_lasso.summary.areaUnderROC)
print("Coefficients: " + str(lrModel_lasso.coefficients))
print("Intercept: " + str(lrModel_lasso.interceptVector))

0.9258607350096711
0.9379947229551452
Training AUROC 0.9258607350096711
Coefficients: (8,[4],[1.9483972754703498])
Intercept: [-2.8502314051542776]

In [0]:
# Create ridge:L2 logistic regression
lr_ridge = LogisticRegression(featuresCol="features", labelCol="top10_trgt", regParam=0.1, elasticNetParam=0)
# Fit the linear regresssion model
lrModel_ridge = lr_ridge.fit(vecTrainDF)
predict_train32 = lrModel_ridge.transform(vecTrainDF)

# Make predictions on testing dataset
predict_test32 = lrModel_ridge.transform(vecTestDF) #make predictions using the trained model

auc_train32 = eval.evaluate(predict_train32)
print(auc_train32)
auc_test32 = eval.evaluate(predict_test32)
print(auc_test32)

print("Training AUROC", lrModel_ridge.summary.areaUnderROC)
print("Coefficients: " + str(lrModel_ridge.coefficients))
print("Intercept: " + str(lrModel_ridge.interceptVector))

0.9834945196647399
0.9942612137203166
Training AUROC 0.9835074145712442
Coefficients: [0.0075282301833286425,0.07260108108517231,0.05331035609868231,-0.1722075107272817,2.197877704650337,0.15058138398646648,0.08169113044369598,-0.03185860273218363]
Intercept: [-5.622715605369718]

In [0]:
# Create elastic net L1+L2 logistic regression
lr_elastnet = LogisticRegression(featuresCol="features", labelCol="top10_trgt", regParam=0.1, elasticNetParam=0.1)
# Fit the regresssion model
lrModel_elastnet = lr_elastnet.fit(vecTrainDF)
predict_train33 = lrModel_elastnet.transform(vecTrainDF)

# Make predictions on testing dataset
predict_test33 = lrModel_elastnet.transform(vecTestDF) #make predictions using the trained model

auc_train33 = eval.evaluate(predict_train33)
print(auc_train33)
auc_test33 = eval.evaluate(predict_test33)
print(auc_test33)

print("Training AUROC", lrModel_elastnet.summary.areaUnderROC)
print("Coefficients: " + str(lrModel_elastnet.coefficients))
print("Intercept: " + str(lrModel_elastnet.interceptVector))

0.9827466150870481
0.9962401055408971
Training AUROC 0.9824629271437778
Coefficients: [0.0038359673970106197,0.06904570746593378,0.007439917295521564,-0.0653108619210248,2.1839926890910917,0.02938418486012431,0.0,0.0]
Intercept: [-5.293275777183871]